# Emitting and Receiving Multiple Events in a Workflow

In this notebook we will explore a simple example of how to emit, process and produce outputs from multiple events at a time.

**1. Install needed dependencies**

In [ ]:
! pip install llama-index-workflows

**2. Create a simple workflow**

We will create a simple workflow that imitates an email-sending automation agent, that can send a batch of emails at once starting from a JSON-like object that maps each contact to the content of the email to send.

We first define the events that are necessary for the workflow steps

In [2]:
from workflows import Workflow, Context, step
from workflows.events import StartEvent, Event, StopEvent
import time
from typing import Dict


class InputEvent(StartEvent):
    emails: Dict[str, str]


class EmailEvent(Event):
    contact: str
    content: str


class ProcessEmailEvent(Event):
    sent: bool
    time: float

Then we define a simple email counter, which will tell us how many emails were sent and how many failed to send. We will pass this as a resource to the workflow.

In [3]:
from workflows.resource import Resource


class EmailCounter:
    def __init__(self):
        self.sent = 0
        self.not_sent = 0

    def update(self, sent: bool):
        if sent:
            self.sent += 1
        else:
            self.not_sent += 1


counter = EmailCounter()


def get_counter(*args, **kwargs):
    return counter

We can finally define the workflow, with a first step that will emit an `EmailEvent` for every contact we have, then a second step that will process each email and return wether that email has been sent or not and at what time it was processed (it will also update the counter), and lastly we will have a step that will collect all the events and will return to us how many emails have failed and how many have succeeded to send.

In [ ]:
import random
from typing import Annotated


class EmailWorkflow(Workflow):
    @step
    async def collect_emails(self, ctx: Context, ev: InputEvent) -> EmailEvent | None:
        events = []
        for contact, content in ev.emails.items():
            events.append(EmailEvent(contact=contact, content=content))
        ctx.send_events(events)
        return

    @step
    async def process_email(
        self,
        ctx: Context,
        ev: EmailEvent,
        counter: Annotated[EmailCounter, Resource(get_counter)],
    ) -> ProcessEmailEvent | None:
        n = random.randint(0, 1)
        print(f"Sending email to {ev.contact}...⌛")
        if n == 1:
            counter.update(sent=True)
            print("Success!✅\n")
            return ProcessEmailEvent(sent=True, time=time.time())
        else:
            print("Failed to send email!❌\n")
            counter.update(sent=False)
            return ProcessEmailEvent(sent=False, time=time.time())

    @step
    async def output(
        self,
        ctx: Context,
        ev: ProcessEmailEvent,
        counter: Annotated[EmailCounter, Resource(get_counter)],
    ) -> StopEvent | None:
        events = ctx.receive_events(ev, [type(ev)])
        if events:
            return None
        return StopEvent(
            result=f"Sent {counter.sent} emails; {counter.not_sent} failed"
        )

**3. Test the workflow**

In [5]:
emails = {
    "john.doe@example.com": "Dear John,\nHow are you?",
    "johanna.doe@example.com": "Dear Johanna,\nHow are you?",
    "mario.rossi@email.it": "Hello Mario!\nHope everything is going weel :)",
    "alice.smith@example.com": "Hi Alice, just checking in.",
    "bob.johnson@email.com": "Meeting notes for tomorrow.",
    "charlie.brown@mail.org": "Your order confirmation.",
    "david.williams@example.net": "Regarding your recent inquiry.",
    "eve.jones@email.co.uk": "Special offer just for you!",
    "frank.miller@mail.net": "Follow up on our conversation.",
    "grace.davis@example.com": "Important update about your account.",
    "heidi.moore@email.org": "Your weekly newsletter.",
    "ivan.taylor@mail.com": "A quick question for you.",
    "judy.anderson@example.net": "Upcoming event details.",
    "kevin.thomas@email.co.uk": "Your monthly statement is ready.",
    "liam.jackson@mail.net": "Exclusive discount code.",
    "mia.white@example.com": "Feedback request for your recent purchase.",
    "noah.harris@email.org": "Invitation to our webinar.",
    "olivia.martin@mail.com": "Happy birthday!",
    "peter.garcia@example.net": "Important security alert.",
    "quinn.rodriguez@email.co.uk": "Your subscription is expiring soon.",
}

In [6]:
wf = EmailWorkflow()
result = await wf.run(start_event=InputEvent(emails=emails))

Sending email to john.doe@example.com...⌛
Success!✅

Sending email to johanna.doe@example.com...⌛
Success!✅

Sending email to mario.rossi@email.it...⌛
Failed to send email!❌

Sending email to alice.smith@example.com...⌛
Failed to send email!❌

Sending email to bob.johnson@email.com...⌛
Failed to send email!❌

Sending email to charlie.brown@mail.org...⌛
Success!✅

Sending email to david.williams@example.net...⌛
Failed to send email!❌

Sending email to eve.jones@email.co.uk...⌛
Failed to send email!❌

Sending email to frank.miller@mail.net...⌛
Success!✅

Sending email to grace.davis@example.com...⌛
Success!✅

Sending email to heidi.moore@email.org...⌛
Success!✅

Sending email to ivan.taylor@mail.com...⌛
Success!✅

Sending email to judy.anderson@example.net...⌛
Success!✅

Sending email to kevin.thomas@email.co.uk...⌛
Success!✅

Sending email to liam.jackson@mail.net...⌛
Success!✅

Sending email to mia.white@example.com...⌛
Success!✅

Sending email to noah.harris@email.org...⌛
Success!✅

S

In [7]:
print(result)

Sent 13 emails; 7 failed
